## Реализовать бинарный классификатор изображений бабочек родов calias и aglais

*  metafile.csv - метафайл всех данных
*  metafile_colias.csv - только colias бабочки
*  metafile_aglais.csv - только aglais бабочки
*  metafile_train.csv - обучающий датасет
*  metafile_test.csv - тестовый датасет

Сами изображения хранятся так:
```
dataset_train/
  aglais/
  colias/
dataset_test/
  aglais/
  colias/
```

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

import os


2024-12-21 16:05:28.734521: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-21 16:05:28.743814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734811528.755444  258965 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734811528.758552  258965 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-21 16:05:28.770950: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [9]:

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

train_dir = 'dataset_train'
test_dir = 'dataset_test'

image_size = (150, 150)
batch_size = 32
epochs = 10

train_datagen = ImageDataGenerator(
    rescale=1./255,
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)


Found 1056 images belonging to 2 classes.
Found 4231 images belonging to 2 classes.


### Настраиваем и компилим модель

In [10]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


/home/v131v/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Обучаем модель на тренировочном датасете

In [11]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)


Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 21s 615ms/step - accuracy: 0.6192 - loss: 0.8307 - val_accuracy: 0.9107 - val_loss: 0.2530
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 20s 614ms/step - accuracy: 0.8920 - loss: 0.2623 - val_accuracy: 0.9347 - val_loss: 0.1951
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 18s 569ms/step - accuracy: 0.9278 - loss: 0.1733 - val_accuracy: 0.9223 - val_loss: 0.2170
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 19s 583ms/step - accuracy: 0.9259 - loss: 0.1583 - val_accuracy: 0.8994 - val_loss: 0.2533
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 19s 585ms/step - accuracy: 0.9331 - loss: 0.1646 - val_accuracy: 0.9179 - val_loss: 0.2737
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 18s 565ms/step - accuracy: 0.9580 - loss: 0.0943 - val_accuracy: 0.9261 - val_loss: 0.2619
Epoch 7/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 19s 585ms/step - accuracy: 0.9699 - loss: 0.0629 - val_accuracy: 0.9280 - val_loss: 0.2683
Epoch 8/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 20s 605ms/step - accuracy: 0.9855 - loss: 0.0421 - val_accu

### Оценка точности

In [12]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Точность на тестовых данных: {test_acc}")


133/133 ━━━━━━━━━━━━━━━━━━━━ 13s 100ms/step - accuracy: 0.9169 - loss: 0.3370
Точность на тестовых данных: 0.9208225011825562


### Сохраняем модельку в keras формате

In [13]:
# Сохранение модели
model.save('butterfly_classifier_without_augmentation.keras')
